In [ ]:
# Run UMAPs, classifiers, and distance metrics
    # on input-data/
    # on mbatch-out/
    # on vae-out/

    # target outdir: ..results/<bnchmrk-phs>/r5/r5-fgs/


In [2]:
%whos

Interactive namespace is empty.


##### rfe

In [5]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

In [ ]:
rfe = RFE(estimator=estimator, n_features_to_select=10, step=10)
file_n = pd.read_csv(pth, sep = '\t', index_col = 0) # read validation split file
X = file_n.iloc[:, 1:]
y = file_n.iloc[:, 0]
rfe.fit(X, y)
mask = rfe.support_
vs_dict[vs] = json.dumps(list(X.columns[mask]))

##### dt and logreg devel

In [6]:
from sklearn import tree
# clf = tree.DecisionTreeClassifier()

In [7]:
from sklearn.linear_model import LogisticRegression
# clf = LogisticRegression(random_state=0).fit(X, y)

#### Imports

In [1]:
# Data handling
import pandas as pd
import glob as glob
import itertools
import time

In [ ]:
# RF
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import statistics
import matplotlib.pyplot as plt

# MMD
import numpy as np
from sklearn.metrics.pairwise import rbf_kernel

# Dstnc, othr
import tensorflow as tf
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import cdist

# UMAP
import umap
import os
from matplotlib.lines import Line2D

#### Start

In [ ]:
systems = 'cell-line+CPTAC' # add beat aml, cptac
modality = 'transcriptomics'

#### Input, mbatch, vae file reads

##### cancers list template

In [ ]:
cancer_types = ['breast-ductal',
    'breast-lobular',
    'breast-nos',
    'colon-adeno',
    'glioblastoma',
    'head-neck',
    'lung-adeno',
    'pancreatic-adeno',
    'renal-clear-cell']

##### r_n

In [4]:
r = 'r5'

##### One cancer file sets

In [15]:
# Set input (raw data) to file set n
phase = 'input-data'
phase_name = 'Input data'
pths_n = sorted(glob.glob('../results/input-data/'+r+'/'+r+'-fls/*'))
print('File count: ', len(pths_n))

File count:  25


'cell-line+BeatAML'

###### mbatch

In [70]:
# Set input (mbatch data) to file set n
phase = 'mbatch-out'
phase_name = 'Mbatch corrected'
pths_n = sorted(glob.glob('../results/mbatch-out/'+r+'/'+r+'-fls/*'))
print('File count: ', len(pths_n))

File count:  13
brain_transcriptomics_HCMI+CPTAC.tsv
 renal-clear-cell_transcriptomics_cell-line+CPTAC.tsv


In [83]:
# Set input (vae data) to file set n
phase = 'vae-out'
phase_name = 'VAE corrected'
pths_n = sorted(glob.glob('../results/vae-out/'+r+'/'+r+'-fls/*.tsv'))
print('File count: ', len(pths_n))

File count:  25


In [ ]:
pths_n

##### One cancer MMD

###### mmd func

In [68]:
def compute_mmd(bio_mrk_1, bio_mrk_2, gamma=.0001):
    K_XX = rbf_kernel(bio_mrk_1, bio_mrk_1, gamma=gamma)
    K_XY = rbf_kernel(bio_mrk_1, bio_mrk_2, gamma=gamma)
    K_YY = rbf_kernel(bio_mrk_2, bio_mrk_2, gamma=gamma)

    m = bio_mrk_1.shape[0]
    n = bio_mrk_2.shape[0]

    mmd = (np.sum(K_XX) - np.trace(K_XX)) / (m * (m - 1))
    mmd += (np.sum(K_YY) - np.trace(K_YY)) / (n * (n - 1))
    mmd -= 2 * np.sum(K_XY) / (m * n)

    # Ensure the MMD value is non-negative
    mmd = np.maximum(mmd, 0)

    return np.sqrt(mmd)

###### MMD main 

In [84]:
print(phase)

# Get distances 
mmd = []
# cos_sim = []
axs_lst = [] # axes means of scipy cdist
c_lst = []
m_lst = []
s_lst = []
for pi, pth_n in enumerate(pths_n):
    print(pi)
    cancer = pth_n.split('/')[-1].split('_')[0]
    modality = pth_n.split('/')[-1].split('_')[1]
    systems = pth_n.split('/')[-1].split('_')[2].split('.')[0]

    c_lst.append(cancer)
    m_lst.append(modality)
    s_lst.append(systems)
    
    df_n = pd.read_csv(
        pth_n, sep = '\t', index_col = 0)
    ab = df_n.System.unique()
    a = df_n[df_n.System == ab[0]]
    b = df_n[df_n.System == ab[1]]
    mmd.append(compute_mmd(a.iloc[:, 2:], b.iloc[:, 2:]))
    axs_lst.append((np.mean(cdist(a.iloc[:, 2:], b.iloc[:, 2:], metric='euclidean'), axis=1).mean(),
                    np.mean(cdist(a.iloc[:, 2:], b.iloc[:, 2:], metric='euclidean'), axis=0).mean()))
    # break
dstncs = pd.DataFrame({'mmd': mmd,
                             'cancer': c_lst,
                             'modality': m_lst,
                             'systems': s_lst,
                             'axs_mns': axs_lst})
# df_n

vae-out
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [87]:
vae_dstncs = dstncs

In [93]:
vae_dstncs.to_csv(
    '../results/stats-out/vae_dstncs_r5.tsv', sep = '\t')

In [ ]:
mbatch_dstncs

##### mbatch

In [75]:
mbatch_dstncs = dstncs

In [92]:
mbatch_dstncs.to_csv(
    '../results/stats-out/mbatch_dstncs_r5.tsv', sep = '\t')

In [73]:
mmd # mbatch, align with cancer type / modality coverage

[0.0,
 0.14927820658906296,
 0.0,
 0.23617940669949763,
 0.24477738090155135,
 0.19255011386163273,
 0.0,
 0.21338705729646135,
 0.0,
 0.0,
 0.32557874415386895,
 0.0,
 0.26118309073188384]

In [72]:
mbatch_mmd_0001 = mmd

##### Input dstncs

In [ ]:
input-data

In [66]:
inpt_axs_lst

[(76.94179143734988, 76.94179143734989),
 (304.5341333159213, 304.5341333159212),
 (16211.721988311636, 16211.721988311638),
 (1863.9358672951798, 1863.9358672951796),
 (708.363443031154, 708.363443031154),
 (16221.288638388125, 16221.288638388127),
 (1619.8829086837588, 1619.8829086837586),
 (693.003878875459, 693.0038788754589),
 (28106.88400184207, 28106.884001842074),
 (19579.504524197335, 19579.50452419734),
 (21002.703583700462, 21002.703583700466),
 (1888.9054863621745, 1888.9054863621745),
 (819.8558001056683, 819.8558001056684),
 (841.3313145573209, 841.3313145573209),
 (19052.4123716267, 19052.412371626702),
 (1844.6023976696295, 1844.602397669629),
 (818.6586597079089, 818.6586597079089),
 (19050.429159943138, 19050.429159943138),
 (14899.49049802119, 14899.490498021192),
 (13834.843774526276, 13834.843774526278),
 (1742.6994203649529, 1742.6994203649524),
 (833.401472101835, 833.4014721018348),
 (20002.578234002573, 20002.578234002573),
 (1846.6127531515694, 1846.6127531515

In [65]:
inpt_axs_lst = axs_lst

In [74]:
input_mmd_0001 = mmd_0001

In [48]:
mmd_0001 = mmd

In [44]:
mmd_001 = mmd

In [40]:
mmd_01 = mmd

In [33]:
input_dstncs = pd.DataFrame({'mmd': mmd, # gamma = 1
                             'cancer': c_lst,
                             'modality': m_lst,
                             'systems': s_lst})

In [89]:
input_dstncs['mmd'] = input_mmd_0001

In [97]:
input_dstncs.to_csv(
    '../results/stats-out/input_dstncs_r5.tsv', sep = '\t')

In [96]:
input_dstncs.head(1)

,mmd,cancer,modality,systems,axs_mns
0,0.485534,AML,proteomics,cell-line+BeatAML,"(76.94179143734988, 76.94179143734989)"


In [95]:
input_dstncs['axs_mns'] = inpt_axs_lst

#### Read stats-out

In [3]:
ls ../results/stats-out/

input_dstncs_r5.tsv   mbatch_dstncs_r5.tsv  vae_dstncs_r5.tsv


In [4]:
so_pths = glob.glob('../results/stats-out/*')

In [5]:
so_pths

['../results/stats-out/vae_dstncs_r5.tsv',
 '../results/stats-out/input_dstncs_r5.tsv',
 '../results/stats-out/mbatch_dstncs_r5.tsv']

In [15]:
df_stck = pd.DataFrame()
for pth_n in so_pths:
    print(pth_n)
    phase = pth_n.split('/')[-1].split('_')[0]
    df_n = pd.read_csv(
        pth_n, sep = '\t', index_col = 0)
    df_n.insert(0, 'phase', phase)
    df_stck = pd.concat([df_stck, df_n], axis = 0)
    # break

../results/stats-out/vae_dstncs_r5.tsv
../results/stats-out/input_dstncs_r5.tsv
../results/stats-out/mbatch_dstncs_r5.tsv


In [18]:
df_stck.iloc[:, :5].to_csv(
    '../results/stats-out/mmd_stack_r5.tsv', sep = '\t')

In [19]:
pth_n = '../results/stats-out/mmd_stack_r5.tsv'
df_stck = pd.read_csv(
        pth_n, sep = '\t', index_col = 0)

In [21]:
df_stck.phase.value_counts()

phase
vae       25
input     25
mbatch    13
Name: count, dtype: int64

In [22]:
df_stck[df_stck.phase == 'mbatch']

,phase,mmd,cancer,modality,systems
0,mbatch,0.000000,brain,transcriptomics,HCMI+CPTAC
1,mbatch,0.149278,breast-lobular,transcriptomics,cell-line+CPTAC
2,mbatch,0.000000,breast,transcriptomics,HCMI+CPTAC
3,mbatch,0.236179,colon-adeno,transcriptomics,cell-line+CPTAC
4,mbatch,0.244777,glioblastoma,transcriptomics,cell-line+CPTAC
5,mbatch,0.192550,head-neck,transcriptomics,cell-line+CPTAC
6,mbatch,0.000000,lung-ad,transcriptomics,HCMI+CPTAC
7,mbatch,0.213387,lung-adeno,transcriptomics,cell-line+CPTAC
8,mbatch,0.000000,lung-sq,transcriptomics,HCMI+CPTAC
9,mbatch,0.000000,ovarian,transcriptomics,HCMI+CPTAC


In [32]:
i_stck = df_stck[df_stck.phase == 'input'].copy()

In [34]:
i_stck['Benchmark'] = i_stck.cancer + '+' + i_stck.modality + '+' + i_stck.systems

In [31]:
v_stck = df_stck[df_stck.phase == 'vae'].copy()

In [33]:
v_stck['Benchmark'] = v_stck.cancer + '+' + v_stck.modality + '+' + v_stck.systems

In [20]:
df_stck

,phase,mmd,cancer,modality,systems
0,vae,0.154165,AML,proteomics,cell-line+BeatAML
1,vae,0.205161,AML,transcriptomics,cell-line+BeatAML
2,vae,0.503925,brain,transcriptomics,CPTAC+HCMI
3,vae,0.997767,breast-ductal,proteomics,cell-line+CPTAC
4,vae,0.450338,breast-lobular,transcriptomics,cell-line+CPTAC
...,...,...,...,...,...
8,mbatch,0.000000,lung-sq,transcriptomics,HCMI+CPTAC
9,mbatch,0.000000,ovarian,transcriptomics,HCMI+CPTAC
10,mbatch,0.325579,pancreatic-adeno,transcriptomics,cell-line+CPTAC
11,mbatch,0.000000,pancreatic,transcriptomics,HCMI+CPTAC


##### dstnc devel

In [49]:
import numpy as np
from scipy.spatial.distance import cdist

In [50]:
points = np.array([[1, 2, 3],
                   [4, 5, 6],
                   [2, 0, 1]])

distances = cdist(points, points, metric='euclidean')
print(distances)

[[0.         5.19615242 3.        ]
 [5.19615242 0.         7.34846923]
 [3.         7.34846923 0.        ]]


In [ ]:
cdist(a.iloc[:, 2:], b.iloc[:, 2:], metric='euclidean')

In [61]:
np.mean(cdist(a.iloc[:, 2:], b.iloc[:, 2:], metric='euclidean'), axis=1).mean()

836.3068844008061

In [63]:
np.mean(cdist(a.iloc[:, 2:], b.iloc[:, 2:], metric='euclidean'), axis=0).mean()

836.3068844008062

In [ ]:
# mean, auc?

In [ ]:
# Pairwise samples or features
tf.keras.losses.cosine_similarity(
    y_true, y_pred, axis=-1
)

##### Pairwise file sets

In [9]:
def get_pairs(file_set_n):
    prot_list = []
    tran_list = []
    for file in file_set_n:
        if 'proteomics' in file:
            prot_list.append(file)
        elif 'transcriptomics' in file:
            tran_list.append(file)
    tran_comb = list(itertools.combinations(tran_list, 2))
    prot_comb = list(itertools.combinations(prot_list, 2))
    return tran_comb, prot_comb

In [78]:
# Set input (raw data) to file set n
phase = 'input-data'
phase_name = 'Input data'

file_set_n = sorted(glob.glob('../strctrd/'+phase+'/'+r+'/'+r+'_fls/*'))
print('File count: ', len(file_set_n))

tran_comb, prot_comb = get_pairs(file_set_n)
print('Trscrptmc, Prtmc comb lens: ', len(tran_comb), len(prot_comb))

# First and last files
print(file_set_n[0].split('/')[-1]+'\n', file_set_n[-1].split('/')[-1])

File count:  15
Trscrptmc, Prtmc comb lens:  28 21
beat-aml_proteomics_cell-line+CPTAC.tsv
 renal-clear-cell_transcriptomics_cell-line+CPTAC.tsv


In [175]:
# mBatch to file set n
phase = 'mbatch-out'
phase_name = 'Mbatch corrected'

file_set_n = sorted(glob.glob('../results/'+phase+'/'+r+'/'+r+'-fls/*'))
print('File count: ', len(file_set_n))

tran_comb, prot_comb = get_pairs(file_set_n)
print('Trscrptmc, Prtmc comb lens: ', len(tran_comb), len(prot_comb))

# First and last files
print(file_set_n[0].split('/')[-1]+'\n', file_set_n[-1].split('/')[-1])

File count:  7
Trscrptmc, Prtmc comb lens:  21 0
breast-lobular_transcriptomics_cell-line+CPTAC.tsv
 renal-clear-cell_transcriptomics_cell-line+CPTAC.tsv


In [10]:
# VAE to file set n
phase = 'vae-out'
phase_name = 'VAE corrected'

file_set_n = sorted(glob.glob('../results/'+phase+'/'+r+'/'+r+'-fls/*'))
print('File count: ', len(file_set_n))

tran_comb, prot_comb = get_pairs(file_set_n)
print('Trscrptmc, Prtmc comb lens: ', len(tran_comb), len(prot_comb))

# First and last files
print(file_set_n[0].split('/')[-1]+'\n', file_set_n[-1].split('/')[-1])

File count:  15
Trscrptmc, Prtmc comb lens:  28 21
beat-aml_proteomics_cell-line+CPTAC_latent_space.tsv
 renal-clear-cell_transcriptomics_cell-line+CPTAC_latent_space.tsv


In [11]:
phase

'vae-out'

##### Infinity vals

In [ ]:
strctrd_n

In [ ]:
strctrd_n.isin([np.inf, -np.inf]).sum().sum()

In [ ]:
inf_count = f"{df_lite.isin([np.inf, -np.inf]).sum().sum():,}"

#### Classifier

##### run 3 n = 9 notes

In [ ]:
# Check
for i, file in enumerate(file_set_n):
    print(strctrd_n)
    break

In [ ]:
file_set_n = ['../results/vae_out/one_cncr/files/breast-ductal_transcriptomics_cell-line+CPTAC_latent_space.tsv',
 '../results/vae_out/one_cncr/files/breast-lobular_transcriptomics_cell-line+CPTAC_latent_space.tsv',
 '../results/vae_out/one_cncr/files/breast-nos_transcriptomics_cell-line+CPTAC_latent_space.tsv',
 # '../results/vae_out/one_cncr/files/colon-adeno_transcriptomics_cell-line+CPTAC_latent_space.tsv',
'../results/vae_out/one_cncr/files/glioblastoma_transcriptomics_cell-line+CPTAC_latent_space.tsv',
 '../results/vae_out/one_cncr/files/head-neck_transcriptomics_cell-line+CPTAC_latent_space.tsv',
 # '../results/vae_out/one_cncr/files/lung-adeno_transcriptomics_cell-line+CPTAC_latent_space.tsv',
 '../results/vae_out/one_cncr/files/pancreatic-adeno_transcriptomics_cell-line+CPTAC_latent_space.tsv',
 '../results/vae_out/one_cncr/files/renal-clear-cell_transcriptomics_cell-line+CPTAC_latent_space.tsv']

##### Clf main

In [ ]:
# Cancer combos n
    # run on three phases: input(../strctrd), mbatch_out, vae_out
start = time.time()

# phase
print(phase_name)

# Transcriptomics toggle
cmb_lst_n = tran_comb

# Proteomics togglea
# cmb_lst_n = prot_comb

# Joint embedding runs
# cmb_lst_n = je_comb
'''
end toggles
'''
for cmb_n, fl_pths in enumerate(cmb_lst_n):
    # print(cmb_n, fl_pths)
    
    strctrd_n1 = pd.read_csv(
        fl_pths[0], sep = '\t', index_col = 0)
    cancer1 = fl_pths[0].split('_')[-3].split('/')[-1]
    print(cancer1)
    
    inf_count = f"{strctrd_n1.isin([np.inf, -np.inf]).sum().sum():,}"
    print(inf_count)

    strctrd_n1.isin([np.inf, -np.inf]).sum().sum()
    
    print(inf_count)
    strctrd_n2 = pd.read_csv(
        fl_pths[1], sep = '\t', index_col = 0)
        
    cancer2 = fl_pths[1].split('_')[-3].split('/')[-1]
    inf_count = f"{strctrd_n2.isin([np.inf, -np.inf]).sum().sum():,}"
    print(inf_count)
    
    # print('check')
    # break

    cancer1 = clf_main(phase, phase_name, fl_pths, strctrd_n1, strctrd_n2)
    # mmd_main(fl_n, file_set_n)
    # umap_main(fl_n, file_set_n)
    # return
    # break
time.time() - start

In [ ]:
strctrd_n1

In [24]:
fl_pths

('../results/vae-out/r4/r4-fls/breast-lobular_transcriptomics_cell-line+CPTAC_latent_space.tsv',
 '../results/vae-out/r4/r4-fls/colon-adeno_transcriptomics_cell-line+CPTAC_latent_space.tsv')

In [22]:
def clf_main(phase, phase_name, fl_pths,
             strctrd_n1, strctrd_n2):

    # print(strctrd.value_counts(strctrd.System))
                # to plot labels

    modality = fl_pths[0].split('_')[-2]
    # return modality
    cancer1 = fl_pths[0].split('_')[-3].split('/')[-1]
    cancer2 = fl_pths[1].split('_')[-3].split('/')[-1]
    # return phase, modality, cancer1, cancer2
    if strctrd_n1.isna().sum().sum() > 0:
        print('strctr1 NaN values: ', cancer1, phase)
        return # To next file
    if strctrd_n2.isin([np.inf, -np.inf]).sum().sum() > 0:
        print('strct2 inf values: ', cancer1, phase)
        return # To next file    

    if strctrd_n2.isna().sum().sum() > 0:
        print('strct2 NaN values: ', cancer2, phase)
        return # To next file
    if strctrd_n2.isin([np.inf, -np.inf]).sum().sum() > 0:
        print('strct2 inf values: ', cancer2, phase)
        return # To next file

    c2_stcr = pd.concat([strctrd_n1, strctrd_n2], axis = 0,
                       join = 'inner')
    X = c2_stcr.iloc[:, 2:]
    y_sys = c2_stcr.System
    y_cncr = c2_stcr.Cancer_type # to Cross val index matching
    # return X
    sys_scores = [] # between these two cancers
    cncr_scores = [] # between these two cancers
    # return fl_pths, cancer1, cancer2
    
    clf_name = 'rnd-frst'
    # clf_name = 'dt'
    # clf_name = 'lg_rg'

    
    for xf_i in list(range(1,10)):
    # for xf_i in list(range(1,30)):
        X_train, X_test, y_train, y_test = train_test_split(
                                                X, y_sys)
        # System                                      ^
        clf = RandomForestClassifier(
                    max_depth=2,
                    random_state=0).fit(
                                    X_train, y_train)
        
        # clf = tree.DecisionTreeClassifier().fit(X_train, y_train)
        # clf = LogisticRegression(random_state=0).fit(X_train, y_train)
        
        score = f1_score(y_test, clf.predict(X_test), average = 'weighted')
        sys_scores.append(score)

        # Cross val index matching           y_c_typ:
        y_train = y_cncr[y_cncr.index.isin(y_train.index)]
        y_test = y_cncr[y_cncr.index.isin(y_test.index)]

        # Cancer type
        clf = RandomForestClassifier(
                    max_depth=2,
                    random_state=0).fit(
                                    X_train, y_train)
        
        score = f1_score(y_test, clf.predict(X_test), average = 'weighted')
        cncr_scores.append(score)
    
    #     break
    # return cncr_scores
    # for score_list in 
    sys_mean = statistics.mean(sys_scores)
    sys_err = statistics.stdev(sys_scores)
    
    fig, ax = plt.subplots()
    plt.ylim(0, 1)
    
    # System barplot
    bar_s = ax.bar('sys', sys_mean, yerr=sys_err, capsize = 7, label='System')
    plt.title(phase_name+' '+cancer1+'+'+cancer2+
        '_'+modality)
    
    # Legend
    ax.legend()
    plt.savefig(
        '../results/'+phase+'/'+r+'/'+r+'_fgs/'+phase+'_'+cancer1+'+'+cancer2+
        '_'+modality+'_'+systems+'_'+clf_name+'_sys.png')
    print(cancer1+'+'+cancer2, modality, ' to disk')
    print(' ')
    plt.close()
    cncr_mean = statistics.mean(cncr_scores)
    cncr_err = statistics.stdev(cncr_scores)
    
    fig, ax = plt.subplots()
    plt.ylim(0, 1)
    
    # Cancer barplot
    bar_c = ax.bar('sys', cncr_mean, yerr=cncr_err, capsize = 7, label='Cancer')
    plt.title(phase_name+' '+cancer1+'+'+cancer2+
        '_'+modality)
    
    # Cancer Legend
    ax.legend()
    plt.savefig(
        '../results/'+phase+'/'+r+'/'+r+'_fgs/'+phase+'_'+cancer1+'+'+cancer2+
        '_'+modality+'_'+systems+'_'+clf_name+'_cncr.png')
    print(cancer1+'+'+cancer2, modality, ' to disk')
    print(' ')
    plt.close()    
    # return x
    # break

#### MMD template

In [ ]:
# order file names by cancer, modality, systems - 
    # for sorting in dirs - mult cncrs per moda, mult modas per system

In [ ]:
def compute_mmd(bio_mrk_1, bio_mrk_2, gamma=1.0):
    K_XX = rbf_kernel(bio_mrk_1, bio_mrk_1, gamma=gamma)
    K_XY = rbf_kernel(bio_mrk_1, bio_mrk_2, gamma=gamma)
    K_YY = rbf_kernel(bio_mrk_2, bio_mrk_2, gamma=gamma)

    m = bio_mrk_1.shape[0]
    n = bio_mrk_2.shape[0]

    mmd = (np.sum(K_XX) - np.trace(K_XX)) / (m * (m - 1))
    mmd += (np.sum(K_YY) - np.trace(K_YY)) / (n * (n - 1))
    mmd -= 2 * np.sum(K_XY) / (m * n)

    # Ensure the MMD value is non-negative
    mmd = np.maximum(mmd, 0)

    return np.sqrt(mmd)

##### mmd inspection

In [ ]:
for i, file in enumerate(file_set_n):
    # print(file)
    strctrd_n = pd.read_csv(
        file,
        sep = '\t', index_col = 0)
    
    # print(strctrd.value_counts(strctrd.System))
    print(' ')
    cancer = cancer_types[i]
    print(cancer)
    # break
    bio_mrk_1 = strctrd_n[strctrd_n.System=='cell_line'].iloc[:, 2:]
    bio_mrk_2 = strctrd_n[strctrd_n.System=='CPTAC'].iloc[:, 2:]
    
    mmd_score = compute_mmd(bio_mrk_1, bio_mrk_2)
    print(mmd_score)
    # break

#### UMAP

In [ ]:
# Set input (raw data) to file set n
phase = 'input_data'
phase_name = 'Input data'
file_set_n = sorted(glob.glob('../strctrd/one_cncr/*'))

In [ ]:
# UMAP on structured data, input data
print(phase_name)
for i, file in enumerate(file_set_n):
    strctrd_n = pd.read_csv(
        file,
        sep = '\t', index_col = 0)
    cancer = cancer_types[i]
    print(cancer)
    print(' ')
    # break
    features = strctrd_n.drop(['System', 'Cancer_type'], axis=1)
    system_labels = strctrd_n['System'] # data phase
    
    umap_result = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, metric='euclidean').fit_transform(features)
    unique_labels = system_labels.unique()
    
    colors = ['red', 'green']
    color_map = dict(zip(unique_labels, colors))
    
    plt.figure(figsize=(10, 8))
    
    for i in range(len(umap_result)):
        plt.scatter(umap_result[i, 0], umap_result[i, 1],
                    c=[color_map[system_labels.iloc[i]]],
                    edgecolor='k',
                    s=100,
                    label=system_labels.iloc[i] + ' - ' + system_labels.iloc[i] if i in [list(system_labels).index(x) for x in unique_labels] else "")
    
    plt.legend()
    
    plt.title(cancer+', '+phase+', '+modality, fontsize=14)
    
    plt.tight_layout()
    plt.savefig(
        '../results/'+phase+'/one_cncr/figures/'+phase+'_'+cancer+
        '_'+modality+'_'+systems+'rf.png')
    
    plt.show()
    # break

In [ ]:
# Set input (mbatch data) to file set n
phase = 'mbatch_out'
phase_name = 'Mbatch corrected'
file_set_n = sorted(glob.glob('../results/mbatch_out/one_cncr/files/*'))

In [ ]:
# UMAP on structured data, mbatch out
print(phase_name)
for i, file in enumerate(file_set_n):
    strctrd_n = pd.read_csv(
        file,
        sep = '\t', index_col = 0)
    cancer = cancer_types[i]
    print(cancer)
    print(' ')
    # break
    features = strctrd_n.drop(['System', 'Cancer_type'], axis=1)
    system_labels = strctrd_n['System'] # data phase
    
    umap_result = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, metric='euclidean').fit_transform(features)
    unique_labels = system_labels.unique()
    
    colors = ['red', 'green']
    color_map = dict(zip(unique_labels, colors))
    
    plt.figure(figsize=(10, 8))
    
    for i in range(len(umap_result)):
        plt.scatter(umap_result[i, 0], umap_result[i, 1],
                    c=[color_map[system_labels.iloc[i]]],
                    edgecolor='k',
                    s=100,
                    label=system_labels.iloc[i] + ' - ' + system_labels.iloc[i] if i in [list(system_labels).index(x) for x in unique_labels] else "")
    
    plt.legend()
    
    plt.title(cancer+', '+phase+', '+modality, fontsize=14)
    
    plt.tight_layout()

    plt.savefig(
        '../results/'+phase+'/one_cncr/figures/'+phase+'_'+cancer+
        '_'+modality+'_'+systems+'rf.png')
    plt.show()
    # break

### UMAP function

In [ ]:
# UMAP
def umap(df, modality, )
    features = df.drop(['Data_phase', 'Labels'], axis=1)
    data_phase = df['Data_phase']
    labels = df['Labels']
    
    umap_result = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, metric='euclidean').fit_transform(features)
    unique_labels = labels.unique()
    
    colors = ['red', 'green']
    color_map = dict(zip(unique_labels, colors))
    
    shape_dict = {'Original': 'o', 'Synthetic': '^'}
    
    # Begin UMAPs on n = 25
    plt.figure(figsize=(10, 8))
    
    for i in range(len(umap_result)):
        plt.scatter(umap_result[i, 0], umap_result[i, 1],
                    c=[color_map[labels.iloc[i]]],
                    marker=shape_dict[data_phase.iloc[i]],
                    edgecolor='k',
                    s=100,
                    label=labels.iloc[i] + ' - ' + data_phase.iloc[i] if i in [list(labels).index(x) for x in unique_labels] else "")
    
    labels = list(unique_labels)
    color_legend = [Line2D([0], [0], marker='s', color='w', label=label,
                           markersize=10, markerfacecolor=color) for label, color in color_map.items() if label in labels]
    
    
    first_legend = plt.legend(handles=color_legend, title='Labels', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.gca().add_artist(first_legend)
    shape_legend = [Line2D([0], [0], marker=shape, color='w', label=status,
                           markersize=10, markerfacecolor='gray') for status, shape in shape_dict.items()]
    plt.legend(handles=shape_legend, title='Data Phase', bbox_to_anchor=(1.05, 0.5), loc='upper left')
    
    plt.title(cancer+', '+data_type, fontsize=14)
    
    plt.tight_layout()
    plt.savefig(f'i_o/UMAP_{v}_{systems}_{cancer}_{modality}.png',
                bbox_inches='tight', dpi = 300) # file name, need to replace cancer name and version with auto vars
    
    plt.show()